# Lluis Badia - SQL Queries using Python

## Importing libraries:

In [71]:
# Importing the necessary libraries:

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass            # password hiding tool
from IPython.display import display

## Odd numbers question:

In [5]:
# Odd numbers:

for x in range(0, 201):
    if "5" not in str(x) and x % 2 == 1:
        print(x)

1
3
7
9
11
13
17
19
21
23
27
29
31
33
37
39
41
43
47
49
61
63
67
69
71
73
77
79
81
83
87
89
91
93
97
99
101
103
107
109
111
113
117
119
121
123
127
129
131
133
137
139
141
143
147
149
161
163
167
169
171
173
177
179
181
183
187
189
191
193
197
199


## Importing CSV files:

In [55]:
# Importing CSV files. Therefore, we can make queries through python:
facturation = pd.read_csv('facturation_test.csv')
algo = pd.read_csv('algo_test.csv', sep = ';')

## Renaming the client id columns from both files:

Therefore, is possible to do a merge (in Python, not in MySQL):

In [56]:
new_f = {'numero_client': 'client_id'}
facturation.rename(columns = new_f, inplace = True)

In [57]:
new_a = {'id': 'client_id'}
algo.rename(columns = new_a, inplace = True)

In [58]:
facturation.columns

Index(['id', 'numero_facture', 'pnr', 'client_id', 'pays', 'date_facturation',
       'activite', 'type', 'montant_vente', 'nom_voyageur', 'description',
       'description_2', 'trajet_origine_destination', 'pays_destination',
       'prestataire', 'date_depart', 'date_retour'],
      dtype='object')

In [59]:
algo.columns

Index(['client_id', 'pays', 'nom_groupe', 'nom_entite'], dtype='object')

In [60]:
# Merging both CSV files:

merged = facturation.merge(algo, on = 'client_id', how = 'inner')


In [61]:
merged.head()

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
0,13096322,F-6200062442,O3W46D,1122,FR,08/02/2018,4,F,225.77,Jean-Jacques|GOLDMAN,Location Voiture ENTERPRISE LYON,LYON,LYON / LYON,FRANCE,ENTERPRISE,04/03/2018,04/04/2018,FR,MICROSOFT,MICROSOFT
1,13096323,F-6200062442,O3W46D,1122,FR,08/02/2018,6,F,0.00,FRAIS RESA VOITURE ONLINE,Emission Location Voiture ENTERPRISE LYON,NaN,LYON / LYON,FRANCE,FRAIS RESA VOITURE ONLINE,04/03/2018,04/04/2018,FR,MICROSOFT,MICROSOFT
2,13097373,F-6200062452,S8TFEH,1122,FR,08/02/2018,4,F,119.99,Johnny|HALLYDAY,Location Voiture ENTERPRISE BORDEAUX,BORDEAUX,BORDEAUX / BORDEAUX,FRANCE,ENTERPRISE,04/11/2018,04/13/2018,FR,MICROSOFT,MICROSOFT
3,13097374,F-6200062452,S8TFEH,1122,FR,08/02/2018,6,F,0.00,FRAIS RESA VOITURE ONLINE,Emission Location Voiture ENTERPRISE BORDEAUX,NaN,BORDEAUX / BORDEAUX,FRANCE,FRAIS RESA VOITURE ONLINE,04/11/2018,04/13/2018,FR,MICROSOFT,MICROSOFT
4,13097375,F-6200062454,Q7626D,1122,FR,08/02/2018,4,F,109.62,Michel|SARDOU,Location Voiture ENTERPRISE PARIS,PARIS,PARIS / PARIS,FRANCE,ENTERPRISE,04/12/2018,04/12/2018,FR,MICROSOFT,MICROSOFT


## MySQL Connection and query testing:

In [2]:
mysql_pw = getpass.getpass()

········


In [4]:
connection_string = 'mysql+pymysql://lbadia:' + mysql_pw + '@127.0.0.1:3306/test_sql'
engine = create_engine(connection_string)

In [8]:
testing = pd.read_sql_query('Select * from facturation_test limit 1', engine)
testing

,id,numero_facture,pnr,numero_client,pays,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour
0,13096322,F-6200062442,O3W46D,1122,FR,2018-08-02,4,F,225.77,Jean-Jacques|GOLDMAN,Location Voiture ENTERPRISE LYON,LYON,LYON / LYON,FRANCE,ENTERPRISE,2018-04-03,2018-04-04


In [9]:
testing2 = pd.read_sql_query('Select * from algo_test limit 1', engine)
testing2

,id,pays,nom_groupe,nom_entite
0,1122,FR,MICROSOFT,MICROSOFT


## Questions:

### Client:

Which SQL request would you do in order to obtain the air expenses in € per Customer group month by months (from invoice date) on the entire data set?

In [16]:
client = pd.read_sql_query("select a.nom_groupe, round(sum(f.montant_vente) * 0.90909, 2) as air_expenses, date_format(f.date_facturation, '%%M') as month from algo_test a inner join facturation_test f on a.id = f.numero_client where f.activite = 5 group by nom_groupe, date_facturation", engine)
client

,nom_groupe,air_expenses,month
0,AFFLELOU,6362.27,August
1,TOYSRUS,6362.27,August
2,NIKE,631.54,August
3,AFFLELOU,3078.77,August
4,TOYSRUS,3078.77,August
...,...,...,...
114,MICROSOFT,153.00,August
115,MICROSOFT,105.40,August
116,MICROSOFT,502.66,August
117,MICROSOFT,145.97,August


In [72]:
# py_client = merged.query('activite == 5')
py_client = merged.groupby(['nom_groupe']).apply(display)
py_client

,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1943,13107691,A-2210000524,UQ2APU,231,FR,08/03/2018,6,A,-50.0,FRAIS TRAVEL 24,INFORMATION : Reference : VR2AQV : 29.3.18 02h04,NaN,/ MEXIQUE,MEXIQUE,FRAIS TRAVEL 24,03/25/2018,03/30/2018,FR,AFFLELOU,AFFLELOU
1945,13107692,A-2210000524,NaN,231,FR,08/03/2018,11,A,50.0,CB Paybox,.,NaN,NaN,MEXIQUE,CB Paybox,08/03/2018,0000-00-00,FR,AFFLELOU,AFFLELOU
1947,13110117,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,747.8,Jacques|DUTRONC,SHENYANG / NANKING,Economy,SHENYANG / NANKING,CHINE,CHINA EASTERN AIRLINES,08/21/2018,08/21/2018,FR,AFFLELOU,AFFLELOU
1949,13110118,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,5951.6,Jacques|DUTRONC,GENEVE / FRANKFURT / SHENYANG - SHANGHAI / FRA...,Economy,GENEVE / SHENYANG,CHINE,LUFTHANSA,08/18/2018,08/24/2018,FR,AFFLELOU,AFFLELOU
1951,13110119,F-2210004550,KCOAM4,231,FR,08/03/2018,6,F,45.0,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,Emission GENEVE / FRANKFURT / SHENYANG - SHANG...,NaN,GENEVE / SHENYANG,CHINE,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,08/18/2018,08/24/2018,FR,AFFLELOU,AFFLELOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,13555053,F-2210010412,LI3IRM,231,FR,09/28/2018,6,F,9.0,FRAIS RESA VOITURE ONLINE,Emission Reservation Voiture HERTZ LONDRES,NaN,LONDRES / LONDRES,ROYAUME-UNI,FRAIS RESA VOITURE ONLINE,10/17/2018,10/19/2018,FR,AFFLELOU,AFFLELOU
3621,13555054,F-2210010412,LHUQA4,231,FR,09/28/2018,6,F,9.0,FRAIS RESA VOITURE ONLINE,Emission Reservation Voiture HERTZ COPENHAGEN,NaN,COPENHAGEN / COPENHAGEN,ROYAUME-UNI,FRAIS RESA VOITURE ONLINE,10/17/2018,10/19/2018,FR,AFFLELOU,AFFLELOU
3623,13591606,F-2210010645,NaN,231,CH,09/26/2018,5,F,9030.2,NaN,GENEVE / FRANKFURT / PHILADELPHIA / FRANKFURT ...,Economy,GENEVE / PHILADELPHIA,ETATS-UNIS,LUFTHANSA,10/03/2018,10/06/2018,FR,AFFLELOU,AFFLELOU
3625,13591607,F-2210010645,NaN,231,CH,09/26/2018,6,F,45.0,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,Emission GENEVE / FRANKFURT / PHILADELPHIA / F...,NaN,GENEVE / PHILADELPHIA,ETATS-UNIS,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,10/03/2018,10/06/2018,FR,AFFLELOU,AFFLELOU


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
0,13096322,F-6200062442,O3W46D,1122,FR,08/02/2018,4,F,225.77,Jean-Jacques|GOLDMAN,Location Voiture ENTERPRISE LYON,LYON,LYON / LYON,FRANCE,ENTERPRISE,04/03/2018,04/04/2018,FR,MICROSOFT,MICROSOFT
1,13096323,F-6200062442,O3W46D,1122,FR,08/02/2018,6,F,0.00,FRAIS RESA VOITURE ONLINE,Emission Location Voiture ENTERPRISE LYON,NaN,LYON / LYON,FRANCE,FRAIS RESA VOITURE ONLINE,04/03/2018,04/04/2018,FR,MICROSOFT,MICROSOFT
2,13097373,F-6200062452,S8TFEH,1122,FR,08/02/2018,4,F,119.99,Johnny|HALLYDAY,Location Voiture ENTERPRISE BORDEAUX,BORDEAUX,BORDEAUX / BORDEAUX,FRANCE,ENTERPRISE,04/11/2018,04/13/2018,FR,MICROSOFT,MICROSOFT
3,13097374,F-6200062452,S8TFEH,1122,FR,08/02/2018,6,F,0.00,FRAIS RESA VOITURE ONLINE,Emission Location Voiture ENTERPRISE BORDEAUX,NaN,BORDEAUX / BORDEAUX,FRANCE,FRAIS RESA VOITURE ONLINE,04/11/2018,04/13/2018,FR,MICROSOFT,MICROSOFT
4,13097375,F-6200062454,Q7626D,1122,FR,08/02/2018,4,F,109.62,Michel|SARDOU,Location Voiture ENTERPRISE PARIS,PARIS,PARIS / PARIS,FRANCE,ENTERPRISE,04/12/2018,04/12/2018,FR,MICROSOFT,MICROSOFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,13598643,F-6200065252,PR8MDA,1122,FR,08/31/2018,1,F,108.30,Ray|CHARLES,PARIS (FR) / LOUVIGNY (FR),NaN,PARIS (FR) / LOUVIGNY (FR),FRANCE,SNCF,08/31/2018,08/31/2018,FR,MICROSOFT,MICROSOFT
1501,13598644,F-6200065252,NaN,1122,FR,08/31/2018,11,F,-108.30,Transfert - Report,.,NaN,NaN,FRANCE,Transfert - Report,10/01/2018,0000-00-00,FR,MICROSOFT,MICROSOFT
1502,13598666,F-6200065525,L2K5YO,1122,FR,08/31/2018,1,F,42.00,Eddy|CLEARWATER,NANCY (FR) / PARIS (FR),NaN,NANCY (FR) / PARIS (FR),FRANCE,SNCF,09/04/2018,09/04/2018,FR,MICROSOFT,MICROSOFT
1503,13598667,F-6200065525,L2K5YO,1122,FR,08/31/2018,6,F,2.76,FRAIS EMIS FER ONLINE,Emission NANCY (FR) / PARIS (FR),NaN,NANCY (FR) / PARIS (FR),FRANCE,FRAIS EMIS FER ONLINE,09/04/2018,09/04/2018,FR,MICROSOFT,MICROSOFT


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1505,13103645,F-9240000055,VN2NEU,7026,FR,08/02/2018,1,F,57.00,Patrick|BRUEL,PARIS (FR) / STRASBOURG (FR),NaN,PARIS (FR) / STRASBOURG (FR),FRANCE,SNCF,09/04/2018,09/04/2018,FR,NIKE,CONVERSE
1506,13103646,F-9240000055,VN2NEU,7026,FR,08/02/2018,1,F,21.10,Patrick|BRUEL,FLEURY LES AUBRAIS (FR) / PARIS (FR),NaN,FLEURY LES AUBRAIS (FR) / PARIS (FR),FRANCE,SNCF,09/04/2018,09/04/2018,FR,NIKE,CONVERSE
1507,13103647,F-9240000055,VN2NEU,7026,FR,08/02/2018,1,F,108.50,Patrick|BRUEL,STRASBOURG (FR) / PARIS (FR) / FLEURY LES AUBR...,NaN,STRASBOURG (FR) / FLEURY LES AUBRAIS (FR),FRANCE,SNCF,09/07/2018,09/07/2018,FR,NIKE,CONVERSE
1508,13103648,F-9240000055,VN2NEU,7026,FR,08/02/2018,6,F,7.20,FRAIS EMIS FER OFFLINE,Emission FLEURY LES AUBRAIS (FR) / PARIS (FR),NaN,FLEURY LES AUBRAIS (FR) / PARIS (FR),FRANCE,FRAIS EMIS FER OFFLINE,09/04/2018,09/04/2018,FR,NIKE,CONVERSE
1509,13103649,F-9240000055,VN2NEU,7026,FR,08/02/2018,6,F,7.20,FRAIS EMIS FER OFFLINE,Emission PARIS (FR) / STRASBOURG (FR),NaN,PARIS (FR) / STRASBOURG (FR),FRANCE,FRAIS EMIS FER OFFLINE,09/04/2018,09/04/2018,FR,NIKE,CONVERSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694,13467326,F-9240002405,VS7KVJ,7024,FR,09/21/2018,6,F,0.00,FRAIS RESA HOTEL OFFLINE,Emission Reservation MERCURE GRANDE MOTTE PORT...,NaN,MONTPELLIER / MONTPELLIER,FRANCE,FRAIS RESA HOTEL OFFLINE,09/26/2018,09/27/2018,FR,NIKE,NIKE
3695,13538253,F-9240002525,J4JFCD,7024,FR,09/27/2018,5,F,401.42,Betty|HUTTON,MARSEILLE / PARIS / MARSEILLE,Economy,MARSEILLE / PARIS,FRANCE,AIR FRANCE,10/02/2018,10/03/2018,FR,NIKE,NIKE
3696,13538254,F-9240002525,J4JFCD,7024,FR,09/27/2018,6,F,10.88,FRAIS EMIS AERIEN NATIONAL OFFLINE,Emission MARSEILLE / PARIS / MARSEILLE,NaN,MARSEILLE / PARIS,FRANCE,FRAIS EMIS AERIEN NATIONAL OFFLINE,10/02/2018,10/03/2018,FR,NIKE,NIKE
3697,13548136,F-9240002554,L64R5Y,7024,FR,09/28/2018,2,F,153.52,Betty|HUTTON,Reservation HOLIDAY INN PARIS CDG AIRPORT,PARIS,PARIS / PARIS,FRANCE,HOLIDAY INN,10/02/2018,10/03/2018,FR,NIKE,NIKE


,id,numero_facture,pnr,client_id,pays_x,date_facturation,activite,type,montant_vente,nom_voyageur,description,description_2,trajet_origine_destination,pays_destination,prestataire,date_depart,date_retour,pays_y,nom_groupe,nom_entite
1942,13107691,A-2210000524,UQ2APU,231,FR,08/03/2018,6,A,-50.0,FRAIS TRAVEL 24,INFORMATION : Reference : VR2AQV : 29.3.18 02h04,NaN,/ MEXIQUE,MEXIQUE,FRAIS TRAVEL 24,03/25/2018,03/30/2018,CH,TOYSRUS,TOYSRUS
1944,13107692,A-2210000524,NaN,231,FR,08/03/2018,11,A,50.0,CB Paybox,.,NaN,NaN,MEXIQUE,CB Paybox,08/03/2018,0000-00-00,CH,TOYSRUS,TOYSRUS
1946,13110117,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,747.8,Jacques|DUTRONC,SHENYANG / NANKING,Economy,SHENYANG / NANKING,CHINE,CHINA EASTERN AIRLINES,08/21/2018,08/21/2018,CH,TOYSRUS,TOYSRUS
1948,13110118,F-2210004550,KCOAM4,231,FR,08/03/2018,5,F,5951.6,Jacques|DUTRONC,GENEVE / FRANKFURT / SHENYANG - SHANGHAI / FRA...,Economy,GENEVE / SHENYANG,CHINE,LUFTHANSA,08/18/2018,08/24/2018,CH,TOYSRUS,TOYSRUS
1950,13110119,F-2210004550,KCOAM4,231,FR,08/03/2018,6,F,45.0,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,Emission GENEVE / FRANKFURT / SHENYANG - SHANG...,NaN,GENEVE / SHENYANG,CHINE,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,08/18/2018,08/24/2018,CH,TOYSRUS,TOYSRUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,13555053,F-2210010412,LI3IRM,231,FR,09/28/2018,6,F,9.0,FRAIS RESA VOITURE ONLINE,Emission Reservation Voiture HERTZ LONDRES,NaN,LONDRES / LONDRES,ROYAUME-UNI,FRAIS RESA VOITURE ONLINE,10/17/2018,10/19/2018,CH,TOYSRUS,TOYSRUS
3620,13555054,F-2210010412,LHUQA4,231,FR,09/28/2018,6,F,9.0,FRAIS RESA VOITURE ONLINE,Emission Reservation Voiture HERTZ COPENHAGEN,NaN,COPENHAGEN / COPENHAGEN,ROYAUME-UNI,FRAIS RESA VOITURE ONLINE,10/17/2018,10/19/2018,CH,TOYSRUS,TOYSRUS
3622,13591606,F-2210010645,NaN,231,CH,09/26/2018,5,F,9030.2,NaN,GENEVE / FRANKFURT / PHILADELPHIA / FRANKFURT ...,Economy,GENEVE / PHILADELPHIA,ETATS-UNIS,LUFTHANSA,10/03/2018,10/06/2018,CH,TOYSRUS,TOYSRUS
3624,13591607,F-2210010645,NaN,231,CH,09/26/2018,6,F,45.0,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,Emission GENEVE / FRANKFURT / PHILADELPHIA / F...,NaN,GENEVE / PHILADELPHIA,ETATS-UNIS,FRAIS EMIS AERIEN LONG COURRIER OFFLINE,10/03/2018,10/06/2018,CH,TOYSRUS,TOYSRUS


""


### Datas Understanding:

Delete all customer payment and charge lines.

In [83]:
delete = merged.drop(merged[merged.activite == 11].index)
delete2 = merged.drop(merged[merged.activite == 6].index)

### Destination Country:

Get the air expenses by destination country and trip in euros for all entities on all period AUGUST, SEPTEMBER 2018.

In [84]:
destination = pd.read_sql_query("select a.nom_entite, round(sum(f.montant_vente) * 0.90909, 2) as air_expenses from algo_test a inner join facturation_test f on a.id = f.numero_client where f.activite = 5 and f.date_facturation between '2018-08-01' and '2018-09-30' group by a.nom_entite", engine)
destination

,nom_entite,air_expenses
0,AFFLELOU,220380.53
1,TOYSRUS,220380.53
2,CONVERSE,8605.03
3,NIKE,2815.32
4,MICROSOFT,8177.68


### Hotel:

Get the Hotel TOP 10 by expenditure in euros from AUGUST 15th 2018, for all customers.

In [85]:
hotel = pd.read_sql_query("select a.id, f.prestataire, round(f.montant_vente * 0.90909, 2) as hotel_exp from algo_test a inner join facturation_test f on a.id = f.numero_client where f.activite = 2 and f.date_facturation = '2018-08-15' order by f.montant_vente desc limit 10", engine)
hotel

,id,prestataire,hotel_exp
0,231,SCANDIC HOTELS,104.79
1,231,SCANDIC HOTELS,104.79
2,231,RADISSON HOTELS,91.32
3,231,RADISSON HOTELS,91.32
4,231,MARRIOTT HTLS RSTS,59.35
5,231,MARRIOTT HTLS RSTS,59.35


### Top 5 of Nike group:

Create a TOP 5 of travelers for the client NIKE in descending order of expenditure over the month of SEPTEMBER 2018.
Add NAME | FIRST NAME of traveler, Number of trips (beware of credit notes) and air expenses.

In [86]:
hotel = pd.read_sql_query("select substring_index(f.nom_voyageur, '|', 1) as name, substring_index(f.nom_voyageur, '|', -1) as surname, f.montant_vente from facturation_test f inner join algo_test a on f.numero_client = a.id where a.nom_groupe = 'NIKE' order by f.montant_vente desc limit 5", engine)
hotel

,name,surname,montant_vente
0,Bing,CROSBY,1001.67
1,Buddy,MILES,1000.94
2,Buddy,MILES,889.73
3,Billy,PAUL,802.45
4,Patrick,BRUEL,699.00


### Potential Saving:

In [87]:
saving = pd.read_sql_query("select case when f.trajet_origine_destination = 'ZURICH / STOCKHOLM' then round(sum(f.montant_vente * 3), 2) else round(sum(f.montant_vente), 2) end as expenses from facturation_test f inner join algo_test a on f.numero_client = a.id where a.nom_groupe = 'AFFLELOU';", engine)
saving

,expenses
0,251934.51
